In [1]:
# =========================
# CELL 1 – QEPC PATH SETUP
# =========================

import sys
from pathlib import Path
import pandas as pd  # <--- add this

# Point to your local QEPC project root
PROJECT_ROOT = Path(r"C:\Users\wdors\qepc_project")

# Make sure the project root is on sys.path so `import qepc...` works
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)

DATA_DIR = PROJECT_ROOT / "data"
CACHE_DIR = PROJECT_ROOT / "cache"

print("DATA_DIR:", DATA_DIR)
print("CACHE_DIR:", CACHE_DIR)


PROJECT_ROOT: C:\Users\wdors\qepc_project
DATA_DIR: C:\Users\wdors\qepc_project\data
CACHE_DIR: C:\Users\wdors\qepc_project\cache


In [2]:
# ==========================================
# CELL 2 – LOAD EOIN (QEPC-READY) DATA
# ==========================================

from qepc.nba.eoin_data_source import (
    load_eoin_games,
    load_eoin_player_boxes,
    load_eoin_team_boxes,
    print_eoin_summary,
)

games_qepc = load_eoin_games()
player_boxes_qepc = load_eoin_player_boxes()
team_boxes_qepc = load_eoin_team_boxes()

print("games_qepc.shape:", games_qepc.shape)
print("player_boxes_qepc.shape:", player_boxes_qepc.shape)
print("team_boxes_qepc.shape:", team_boxes_qepc.shape)

# Quick peek at the first few rows
print("\nGames sample:")
display(games_qepc.head())

print("\nPlayer boxes sample:")
display(player_boxes_qepc.head())

print("\nTeam boxes sample:")
display(team_boxes_qepc.head())

# Optional summary helper
print_eoin_summary(games_qepc, player_boxes_qepc, team_boxes_qepc)


games_qepc.shape: (72290, 19)
player_boxes_qepc.shape: (1638878, 36)
team_boxes_qepc.shape: (144580, 49)

Games sample:


,game_id,game_datetime,home_team_city,home_team_name,home_team_id,away_team_city,away_team_name,away_team_id,home_score,away_score,winner_team_id,gametype,attendance,arenaid,gamelabel,gamesublabel,seriesgamenumber,game_date,is_final
0,22500349,2025-12-05 16:30:00+00:00,Oklahoma City,Thunder,1610612760,Dallas,Mavericks,1610612742,132,111,1610612760,None,18203.0,NaN,None,None,NaN,2025-12-05,True
1,22500345,2025-12-05 15:00:00+00:00,Chicago,Bulls,1610612741,Indiana,Pacers,1610612754,105,120,1610612754,None,20471.0,NaN,None,None,NaN,2025-12-05,True
2,22500347,2025-12-05 15:00:00+00:00,Memphis,Grizzlies,1610612763,LA,Clippers,1610612746,107,98,1610612763,None,15052.0,NaN,None,None,NaN,2025-12-05,True
3,22500348,2025-12-05 15:00:00+00:00,Milwaukee,Bucks,1610612749,Philadelphia,76ers,1610612755,101,116,1610612755,None,17341.0,NaN,None,None,NaN,2025-12-05,True
4,22500346,2025-12-05 15:00:00+00:00,Houston,Rockets,1610612745,Phoenix,Suns,1610612756,117,98,1610612745,None,18055.0,NaN,None,None,NaN,2025-12-05,True



Player boxes sample:


,firstname,lastname,player_id,game_id,game_datetime,team_city,team_name,opp_team_city,opp_team_name,gametype,...,freethrowsattempted,freethrowsmade,freethrowspercentage,reboundsdefensive,reboundsoffensive,reboundstotal,foulspersonal,turnovers,plusminuspoints,game_date
0,D'Angelo,Russell,1626156,22500349,2025-12-05 16:30:00+00:00,Dallas,Mavericks,Oklahoma City,Thunder,None,...,0.0,0.0,0.000,3.0,0.0,3.0,2.0,1.0,-3.0,2025-12-05
1,Shai,Gilgeous-Alexander,1628983,22500349,2025-12-05 16:30:00+00:00,Oklahoma City,Thunder,Dallas,Mavericks,None,...,12.0,11.0,0.917,5.0,0.0,5.0,4.0,2.0,24.0,2025-12-05
2,Caleb,Martin,1628997,22500349,2025-12-05 16:30:00+00:00,Dallas,Mavericks,Oklahoma City,Thunder,None,...,0.0,0.0,0.000,0.0,0.0,0.0,1.0,0.0,-2.0,2025-12-05
3,Kenrich,Williams,1629026,22500349,2025-12-05 16:30:00+00:00,Oklahoma City,Thunder,Dallas,Mavericks,None,...,2.0,2.0,1.000,0.0,1.0,1.0,0.0,0.0,-3.0,2025-12-05
4,Naji,Marshall,1630230,22500349,2025-12-05 16:30:00+00:00,Dallas,Mavericks,Oklahoma City,Thunder,None,...,4.0,2.0,0.500,1.0,0.0,1.0,3.0,0.0,-24.0,2025-12-05



Team boxes sample:


,game_id,game_datetime,team_city,team_name,team_id,opp_team_city,opp_team_name,opp_team_id,home,win,...,pointsfastbreak,pointsfromturnovers,pointsinthepaint,pointssecondchance,timestied,timeoutsremaining,seasonwins,seasonlosses,coachid,game_date
0,22500349,2025-12-05 16:30:00+00:00,Dallas,Mavericks,1610612742,Oklahoma City,Thunder,1610612760,0,0,...,12.0,8.0,38.0,8.0,8.0,1.0,8.0,16.0,NaN,2025-12-05
1,22500349,2025-12-05 16:30:00+00:00,Oklahoma City,Thunder,1610612760,Dallas,Mavericks,1610612742,1,1,...,10.0,18.0,62.0,9.0,8.0,1.0,22.0,1.0,NaN,2025-12-05
2,22500345,2025-12-05 15:00:00+00:00,Chicago,Bulls,1610612741,Indiana,Pacers,1610612754,1,0,...,8.0,13.0,36.0,18.0,4.0,0.0,9.0,13.0,NaN,2025-12-05
3,22500346,2025-12-05 15:00:00+00:00,Houston,Rockets,1610612745,Phoenix,Suns,1610612756,1,1,...,16.0,21.0,66.0,8.0,7.0,1.0,15.0,5.0,NaN,2025-12-05
4,22500347,2025-12-05 15:00:00+00:00,LA,Clippers,1610612746,Memphis,Grizzlies,1610612763,0,0,...,13.0,23.0,36.0,9.0,9.0,1.0,6.0,17.0,NaN,2025-12-05


=== Eoin / QEPC Data Summary ===
Games:            72290 rows, 19 columns
  game_datetime: 1946-11-26 19:00:00+00:00  →  2025-12-05 16:30:00+00:00
Player boxes:   1638878 rows, 36 columns
  game_datetime: 1946-11-26 19:00:00+00:00  →  2025-12-05 16:30:00+00:00
Team boxes:      144580 rows, 49 columns
  game_datetime: 1946-11-26 19:00:00+00:00  →  2025-12-05 16:30:00+00:00
Max season record seen in team_boxes: 68.0–65.0 (approx)


In [3]:
# ==========================================
# CELL 3 – TEAM STATS & TEAM STRENGTHS
# ==========================================

from qepc.nba.eoin_team_stats import (
    build_team_stats_from_eoin,
    save_team_stats_to_cache,
)
from qepc.nba.team_strengths_eoin import (
    calculate_advanced_strengths_from_eoin,
    save_advanced_strengths_to_cache,
)

# Build team_stats from Eoin team boxes
team_stats = build_team_stats_from_eoin(team_boxes_qepc)

print("team_stats.shape:", team_stats.shape)
print("\nteam_stats sample:")
display(team_stats.head())

# Save to cache (parquet)
team_stats_path = save_team_stats_to_cache(team_stats)
print("\nSaved team_stats to:", team_stats_path)

# Build strengths from team_stats
team_strengths = calculate_advanced_strengths_from_eoin(team_stats, verbose=True)

print("\nteam_strengths.shape:", team_strengths.shape)
print("\nteam_strengths (top 10 by strength_score):")
display(
    team_strengths[
        [
            "team_id",
            "games_played",
            "win_pct",
            "off_ppg",
            "def_ppg",
            "pts_diff_per_game",
            "strength_score",
            "strength_rank",
        ]
    ].head(10)
)

# Save strengths to cache
team_strengths_path = save_advanced_strengths_to_cache(team_strengths)
print("\nSaved team_strengths to:", team_strengths_path)


team_stats.shape: (34, 14)

team_stats sample:


,team_id,games_played,wins,losses,win_pct,pts_for,pts_against,pts_diff,off_ppg,def_ppg,reb_total,reb_pg,ast_total,ast_pg
0,15016,1,0,1,0.000000,97,107,-10,97.000000,107.000000,48.0,48.000000,23.0,23.000000
1,15018,3,0,3,0.000000,257,395,-138,85.666667,131.666667,80.0,26.666667,49.0,16.333333
2,50013,1,0,1,0.000000,92,127,-35,92.000000,127.000000,42.0,42.000000,24.0,24.000000
3,50014,1,0,1,0.000000,88,123,-35,88.000000,123.000000,38.0,38.000000,18.0,18.000000
4,1610612737,6462,3163,3299,0.489477,668300,671894,-3594,103.419994,103.976168,174313.0,26.975085,87465.0,13.535283


Saved team_stats to: C:\Users\wdors\qepc_project\cache\imports\eoin_team_stats.parquet

Saved team_stats to: C:\Users\wdors\qepc_project\cache\imports\eoin_team_stats.parquet
Built advanced strengths from Eoin team_stats:
      team_id  games_played   win_pct     off_ppg     def_ppg  \
0  1610612738          6883  0.596833  106.253523  103.205434   
1  1610612747          6898  0.585242  106.762540  104.581328   
2  1610612759          4492  0.585485  104.987311  102.372663   
3  1610612756          5074  0.532913  107.760347  106.615491   
4  1610612760          5192  0.539676  105.905817  104.664291   
5  1610612749          5075  0.521773  105.209064  104.075862   
6  1610612743          4360  0.505963  108.001376  108.068349   
7  1610612745          5160  0.518411  105.497093  104.875000   
8  1610612762          4552  0.527900  103.588752  102.658172   
9  1610612757          4865  0.515313  105.058582  104.502980   

   pts_diff_per_game  strength_score  strength_rank  
0       

,team_id,games_played,win_pct,off_ppg,def_ppg,pts_diff_per_game,strength_score,strength_rank
0,1610612738,6883,0.596833,106.253523,103.205434,3.048089,0.779025,1
1,1610612747,6898,0.585242,106.762540,104.581328,2.181212,0.728369,2
2,1610612759,4492,0.585485,104.987311,102.372663,2.614648,0.700651,3
3,1610612756,5074,0.532913,107.760347,106.615491,1.144856,0.584849,4
4,1610612760,5192,0.539676,105.905817,104.664291,1.241525,0.556911,5
5,1610612749,5075,0.521773,105.209064,104.075862,1.133202,0.490957,6
6,1610612743,4360,0.505963,108.001376,108.068349,-0.066972,0.475774,7
7,1610612745,5160,0.518411,105.497093,104.875000,0.622093,0.469176,8
8,1610612762,4552,0.527900,103.588752,102.658172,0.930580,0.455196,9
9,1610612757,4865,0.515313,105.058582,104.502980,0.555601,0.447508,10


Saved advanced strengths to: C:\Users\wdors\qepc_project\cache\imports\eoin_team_strengths.parquet

Saved team_strengths to: C:\Users\wdors\qepc_project\cache\imports\eoin_team_strengths.parquet


In [4]:
# ==========================================================
# CELL 4 – MATCHUPS FOR A DATE + TEAM PTS/REB/AST EXPECTED
# ==========================================================

from qepc.nba.matchups_eoin import build_matchups_for_date, MatchupConfig
from qepc.nba.team_scoring_eoin import attach_scoring_predictions, ScoringConfig

# Choose a date you know has games in Games.csv
TARGET_DATE = "2025-12-05"

# Include final games for sanity/backtests
matchup_config = MatchupConfig(include_final_games=True)
matchups = build_matchups_for_date(TARGET_DATE, config=matchup_config)

print(f"Matchups on {TARGET_DATE}: {len(matchups)} games")
display(
    matchups[
        [
            "game_id",
            "game_date",
            "game_datetime",
            "home_team_name",
            "away_team_name",
            "home_team_id",
            "away_team_id",
            "home_strength_score",
            "away_strength_score",
        ]
    ].head(20)
)

# Attach expected points / rebounds / assists for each team
scoring_config = ScoringConfig(
    home_court_advantage=1.5,
    weight_off_edge=0.7,
    weight_def_edge=0.7,
    # REB/AST weights can stay defaults for now
)

matchups_with_scores = attach_scoring_predictions(matchups, config=scoring_config)

print("\nColumns with expected stats (exp_*):")
print([c for c in matchups_with_scores.columns if c.startswith("exp_")])

print("\nMatchups with team expectations sample:")
display(
    matchups_with_scores[
        [
            "game_id",
            "game_date",
            "home_team_name",
            "away_team_name",
            "exp_home_pts",
            "exp_away_pts",
            "exp_home_reb",
            "exp_away_reb",
            "exp_home_ast",
            "exp_away_ast",
        ]
    ].head(20)
)


Matchups on 2025-12-05: 12 games


,game_id,game_date,game_datetime,home_team_name,away_team_name,home_team_id,away_team_id,home_strength_score,away_strength_score
0,22500339,2025-12-05,2025-12-05 14:00:00+00:00,Magic,Heat,1610612753,1610612748,0.196416,0.329388
1,22500338,2025-12-05,2025-12-05 14:00:00+00:00,Celtics,Lakers,1610612738,1610612747,0.779025,0.728369
2,22500341,2025-12-05,2025-12-05 14:30:00+00:00,Cavaliers,Spurs,1610612739,1610612759,0.204271,0.700651
3,22500343,2025-12-05,2025-12-05 14:30:00+00:00,Knicks,Jazz,1610612752,1610612762,0.274537,0.455196
4,22500340,2025-12-05,2025-12-05 14:30:00+00:00,Hawks,Nuggets,1610612737,1610612743,0.296308,0.475774
5,22500344,2025-12-05,2025-12-05 14:30:00+00:00,Raptors,Hornets,1610612761,1610612766,0.229671,-0.028240
6,22500342,2025-12-05,2025-12-05 14:30:00+00:00,Pistons,Trail Blazers,1610612765,1610612757,0.220679,0.447508
7,22500345,2025-12-05,2025-12-05 15:00:00+00:00,Bulls,Pacers,1610612741,1610612754,0.366119,0.352127
8,22500347,2025-12-05,2025-12-05 15:00:00+00:00,Grizzlies,Clippers,1610612763,1610612746,0.025832,0.075263
9,22500348,2025-12-05,2025-12-05 15:00:00+00:00,Bucks,76ers,1610612749,1610612755,0.490957,0.428929



Columns with expected stats (exp_*):
['exp_home_pts', 'exp_away_pts', 'exp_home_reb', 'exp_away_reb', 'exp_home_ast', 'exp_away_ast']

Matchups with team expectations sample:


,game_id,game_date,home_team_name,away_team_name,exp_home_pts,exp_away_pts,exp_home_reb,exp_away_reb,exp_home_ast,exp_away_ast
0,22500339,2025-12-05,Magic,Heat,109.787019,105.268142,39.096112,38.346505,20.600310,20.210821
1,22500338,2025-12-05,Celtics,Lakers,109.492322,109.311760,30.747403,30.777525,16.762972,16.642307
2,22500341,2025-12-05,Cavaliers,Spurs,107.780371,108.673197,34.671308,36.623736,18.802474,19.879764
3,22500343,2025-12-05,Knicks,Jazz,108.316113,107.209889,30.489426,35.841800,16.266460,19.840483
4,22500340,2025-12-05,Hawks,Nuggets,105.067937,109.639431,31.213371,36.878237,16.400656,20.129115
5,22500344,2025-12-05,Raptors,Hornets,107.137849,105.839621,38.718128,38.385503,20.798303,20.977721
6,22500342,2025-12-05,Pistons,Trail Blazers,106.871364,108.088979,30.788791,34.705993,16.353421,18.383682
7,22500345,2025-12-05,Bulls,Pacers,107.969677,107.579364,33.905362,36.479685,18.446368,19.728952
8,22500347,2025-12-05,Grizzlies,Clippers,103.971856,108.172461,38.833376,34.232865,20.705656,18.339439
9,22500348,2025-12-05,Bucks,76ers,108.772511,107.356289,33.960902,30.429492,18.596561,16.165813


In [5]:
# ===============================================
# CELL 5 – PLAYER USAGE (PTS / REB / AST SHARES)
# ===============================================

from qepc.nba.player_usage_eoin import build_player_usage_from_eoin

player_usage = build_player_usage_from_eoin(
    player_boxes=player_boxes_qepc,
    min_games=5,                 # tune as you like
    cutoff_date="2024-10-01",    # keep it recent
    use_recency_weights=True,    # <-- NEW: turn decoherence on
    tau_points_days=30.0,        # coherence time for scoring
    # tau_rebounds_days=None,    # None -> same as points
    # tau_assists_days=None,     # None -> same as points
)

print("player_usage.shape:", player_usage.shape)

display(
    player_usage[
        [
            "player_id",
            "player_name",
            "team_name",
            "games_played",
            "avg_points",
            "avg_rebounds",
            "avg_assists",
            "mean_points_share",
            "mean_rebounds_share",
            "mean_assists_share",
        ]
    ].head(20)
)


player_usage.shape: (894, 13)


,player_id,player_name,team_name,games_played,avg_points,avg_rebounds,avg_assists,mean_points_share,mean_rebounds_share,mean_assists_share
0,2544,LeBron James,Lakers,90,14.424793,4.608175,6.948620,0.120850,0.115957,0.268296
1,101108,Chris Paul,Clippers,24,2.788335,1.395722,2.559663,0.024153,0.033523,0.103174
2,101108,Chris Paul,Spurs,84,8.630952,3.476190,7.214286,0.075845,0.077530,0.253688
3,200768,Kyle Lowry,76ers,67,0.522019,0.184684,0.376560,0.004542,0.004504,0.014868
4,200782,P.J. Tucker,Knicks,35,0.257143,0.228571,0.000000,0.002326,0.006471,0.000000
5,201142,Kevin Durant,Rockets,21,24.582928,4.937264,4.400836,0.200812,0.102516,0.177038
6,201142,Kevin Durant,Suns,67,25.537313,5.850746,4.134328,0.224922,0.136017,0.143967
7,201143,Al Horford,Celtics,87,7.218391,5.022989,1.712644,0.062607,0.112724,0.070970
8,201143,Al Horford,Warriors,17,5.504705,4.365550,2.311957,0.047418,0.107632,0.080497
9,201144,Mike Conley,Timberwolves,117,5.324298,1.767819,3.246909,0.044228,0.041973,0.120212


In [6]:
# ============================================================
# CELL 6 – PLAYER PTS/REB/AST EXPECTATIONS FOR ONE MATCHUP
# ============================================================

import numpy as np

# Try to find a specific matchup (Celtics vs Lakers) on that date
mask_bos_lal = (
    (matchups_with_scores["home_team_name"] == "Celtics")
    & (matchups_with_scores["away_team_name"] == "Lakers")
)

if matchups_with_scores[mask_bos_lal].empty:
    # Fallback: just take the first game on the slate
    example_game = matchups_with_scores.iloc[0]
    print("Could not find Celtics-Lakers; using first game on slate instead.")
else:
    example_game = matchups_with_scores[mask_bos_lal].iloc[0]
    print("Using Celtics-Lakers game.")

display(
    example_game[
        [
            "game_id",
            "game_date",
            "home_team_name",
            "away_team_name",
            "exp_home_pts",
            "exp_away_pts",
            "exp_home_reb",
            "exp_away_reb",
            "exp_home_ast",
            "exp_away_ast",
        ]
    ].to_frame().T
)

home_team = example_game["home_team_name"]
away_team = example_game["away_team_name"]

home_pts_lambda = float(example_game["exp_home_pts"])
away_pts_lambda = float(example_game["exp_away_pts"])

home_reb_lambda = example_game.get("exp_home_reb", np.nan)
away_reb_lambda = example_game.get("exp_away_reb", np.nan)

home_ast_lambda = example_game.get("exp_home_ast", np.nan)
away_ast_lambda = example_game.get("exp_away_ast", np.nan)

print("\nTeam-level expectations:")
print(f"Home {home_team}: λ_pts={home_pts_lambda:.2f}, λ_reb={home_reb_lambda}, λ_ast={home_ast_lambda}")
print(f"Away {away_team}: λ_pts={away_pts_lambda:.2f}, λ_reb={away_reb_lambda}, λ_ast={away_ast_lambda}")


Using Celtics-Lakers game.


,game_id,game_date,home_team_name,away_team_name,exp_home_pts,exp_away_pts,exp_home_reb,exp_away_reb,exp_home_ast,exp_away_ast
1,22500338,2025-12-05,Celtics,Lakers,109.492322,109.31176,30.747403,30.777525,16.762972,16.642307



Team-level expectations:
Home Celtics: λ_pts=109.49, λ_reb=30.74740341274342, λ_ast=16.7629721092934
Away Lakers: λ_pts=109.31, λ_reb=30.777524661885195, λ_ast=16.642306790649137


In [7]:
# ==========================================================
# CELL 6A – COMPARE UNWEIGHTED VS RECENCY-WEIGHTED USAGE
# ==========================================================

from qepc.nba.player_usage_eoin import build_player_usage_from_eoin

# 1) Old-school usage: no recency weights
player_usage_flat = build_player_usage_from_eoin(
    player_boxes=player_boxes_qepc,
    min_games=5,
    cutoff_date=None,            # use full history
    use_recency_weights=False,
)

# 2) New quantum usage: recency-weighted
player_usage_rw = player_usage  # from Cell 5 (already recency-weighted)

# Align on (player_id, team_name)
cols_key = ["player_id", "team_name"]
cols_keep = [
    "player_id", "team_name", "player_name",
    "avg_points", "avg_rebounds", "avg_assists",
]

flat = player_usage_flat[cols_keep].rename(
    columns={
        "avg_points": "flat_avg_points",
        "avg_rebounds": "flat_avg_rebounds",
        "avg_assists": "flat_avg_assists",
    }
)

rw = player_usage_rw[cols_keep].rename(
    columns={
        "avg_points": "rw_avg_points",
        "avg_rebounds": "rw_avg_rebounds",
        "avg_assists": "rw_avg_assists",
    }
)

merged = flat.merge(rw, on=["player_id", "team_name", "player_name"], how="inner")

# Show players where recency moved points the most
merged["delta_points"] = merged["rw_avg_points"] - merged["flat_avg_points"]

merged_sorted = merged.sort_values("delta_points", ascending=False)

print("Players whose scoring average moved UP the most with recency weights:")
display(merged_sorted.head(15))

print("Players whose scoring average moved DOWN the most with recency weights:")
display(merged_sorted.tail(15))


Players whose scoring average moved UP the most with recency weights:


,player_id,team_name,player_name,flat_avg_points,flat_avg_rebounds,flat_avg_assists,rw_avg_points,rw_avg_rebounds,rw_avg_assists,delta_points
473,1630559,Lakers,Austin Reaves,14.355795,3.708895,4.102426,27.244385,5.192888,5.937469,12.888589
465,1630552,Hawks,Jalen Johnson,9.238908,5.051195,2.303754,20.254854,9.432298,7.048785,11.015946
381,1630178,76ers,Tyrese Maxey,18.203661,2.846682,4.048055,29.175536,4.268671,6.232327,10.971874
584,1631157,Bucks,Ryan Rollins,7.084746,1.847458,2.177966,16.303768,3.841712,5.494876,9.219022
396,1630202,Celtics,Payton Pritchard,8.455268,2.457256,2.415507,17.028241,4.130045,4.636839,8.572973
777,1642281,Cavaliers,Jaylon Tyson,4.587156,2.100917,0.944954,12.810847,5.067330,1.799287,8.223691
367,1630166,Trail Blazers,Deni Avdija,17.018018,6.513514,4.000000,24.934327,7.462370,6.387376,7.916309
678,1641730,Nets,Noah Clowney,7.380952,3.055556,0.896825,14.706045,3.598516,1.786629,7.325092
496,1630595,Pistons,Cade Cunningham,19.621429,4.878571,6.621429,26.855288,6.456406,8.612763,7.233860
759,1642263,Rockets,Reed Sheppard,5.052632,1.491228,1.438596,12.002623,3.276486,3.080322,6.949992


Players whose scoring average moved DOWN the most with recency weights:


,player_id,team_name,player_name,flat_avg_points,flat_avg_rebounds,flat_avg_assists,rw_avg_points,rw_avg_rebounds,rw_avg_assists,delta_points
189,1628368,Kings,De'Aaron Fox,NaN,NaN,NaN,23.740000,4.960000,5.760000,NaN
191,1628369,Celtics,Jayson Tatum,NaN,NaN,NaN,17.480676,5.811186,3.887212,NaN
193,1628371,Magic,Jonathan Isaac,NaN,NaN,NaN,3.194917,3.268542,0.152448,NaN
205,1628389,Heat,Bam Adebayo,NaN,NaN,NaN,18.924521,9.385856,2.570878,NaN
224,1628449,Raptors,Chris Boucher,NaN,NaN,NaN,6.722892,2.927711,0.433735,NaN
227,1628467,Mavericks,Maxi Kleber,NaN,NaN,NaN,2.575000,2.375000,1.075000,NaN
236,1628966,Timberwolves,Keita Bates-Diop,NaN,NaN,NaN,2.000000,1.000000,0.000000,NaN
238,1628970,Hornets,Miles Bridges,NaN,NaN,NaN,19.857995,6.162984,3.149531,NaN
251,1628991,Grizzlies,Jaren Jackson Jr.,NaN,NaN,NaN,16.983266,4.748160,1.870528,NaN
266,1629008,Nuggets,Michael Porter Jr.,NaN,NaN,NaN,16.736842,6.694737,1.852632,NaN


In [8]:
# =====================================================
# CELL 7 – HOME TEAM PLAYER PTS/REB/AST PROJECTIONS
# =====================================================

home_usage = player_usage[player_usage["team_name"] == home_team].copy()

# Expected points
home_usage["exp_points"] = home_pts_lambda * home_usage["mean_points_share"]

# Expected rebounds (if we have both the team λ and share)
if not pd.isna(home_reb_lambda) and "mean_rebounds_share" in home_usage.columns:
    home_usage["exp_rebounds"] = home_reb_lambda * home_usage["mean_rebounds_share"]

# Expected assists
if not pd.isna(home_ast_lambda) and "mean_assists_share" in home_usage.columns:
    home_usage["exp_assists"] = home_ast_lambda * home_usage["mean_assists_share"]

# Sort by expected points descending for now
home_usage = home_usage.sort_values("exp_points", ascending=False)

cols = [
    "player_name",
    "games_played",
    "avg_points",
    "avg_rebounds",
    "avg_assists",
    "mean_points_share",
    "mean_rebounds_share",
    "mean_assists_share",
    "exp_points",
    "exp_rebounds",
    "exp_assists",
]

available_cols = [c for c in cols if c in home_usage.columns]

print(f"Home team projections – {home_team}")
display(home_usage[available_cols].head(15))


Home team projections – Celtics


,player_name,games_played,avg_points,avg_rebounds,avg_assists,mean_points_share,mean_rebounds_share,mean_assists_share,exp_points,exp_rebounds,exp_assists
170,Jaylen Brown,113,27.801720,6.245412,5.073245,0.237364,0.142244,0.211414,25.989585,4.373623,3.543930
191,Jayson Tatum,96,17.480676,5.811186,3.887212,0.151075,0.129758,0.152251,16.541509,3.989730,2.552179
210,Derrick White,120,17.890551,4.308623,5.351259,0.148732,0.097968,0.209337,16.284979,3.012266,3.509114
396,Payton Pritchard,124,17.028241,4.130045,4.636839,0.142830,0.094001,0.179182,15.638803,2.890299,3.003621
121,Kristaps Porzingis,69,13.086957,4.855072,1.376812,0.114580,0.106081,0.052068,12.545609,3.261722,0.872811
271,Anfernee Simons,27,13.049378,2.201868,2.650990,0.110018,0.050101,0.106879,12.046095,1.540465,1.791604
350,Neemias Queta,123,9.490047,7.103858,1.340037,0.079268,0.161173,0.050113,8.679288,4.955664,0.840035
25,Jrue Holiday,86,9.093023,3.534884,3.174419,0.078590,0.078907,0.127234,8.605042,2.426182,2.132814
7,Al Horford,87,7.218391,5.022989,1.712644,0.062607,0.112724,0.070970,6.854967,3.465971,1.189673
588,Josh Minott,27,7.410514,4.494917,1.244385,0.059894,0.102916,0.046173,6.557932,3.164415,0.774003


In [9]:
# =====================================================
# CELL 8 – AWAY TEAM PLAYER PTS/REB/AST PROJECTIONS
# =====================================================

away_usage = player_usage[player_usage["team_name"] == away_team].copy()

away_usage["exp_points"] = away_pts_lambda * away_usage["mean_points_share"]

if not pd.isna(away_reb_lambda) and "mean_rebounds_share" in away_usage.columns:
    away_usage["exp_rebounds"] = away_reb_lambda * away_usage["mean_rebounds_share"]

if not pd.isna(away_ast_lambda) and "mean_assists_share" in away_usage.columns:
    away_usage["exp_assists"] = away_ast_lambda * away_usage["mean_assists_share"]

away_usage = away_usage.sort_values("exp_points", ascending=False)

home_usage["exp_PRA"] = (
    home_usage["exp_points"].fillna(0)
    + home_usage["exp_rebounds"].fillna(0)
    + home_usage["exp_assists"].fillna(0)
)

cols = [
    "player_name",
    "games_played",
    "avg_points",
    "avg_rebounds",
    "avg_assists",
    "mean_points_share",
    "mean_rebounds_share",
    "mean_assists_share",
    "exp_points",
    "exp_rebounds",
    "exp_assists",
]

available_cols = [c for c in cols if c in away_usage.columns]


print(f"Away team projections – {away_team}")
display(away_usage[available_cols].head(15))


Away team projections – Lakers


,player_name,games_played,avg_points,avg_rebounds,avg_assists,mean_points_share,mean_rebounds_share,mean_assists_share,exp_points,exp_rebounds,exp_assists
283,Luka Doncic,53,32.702220,8.190821,8.239232,0.273814,0.199334,0.329364,29.931036,6.135017,5.481376
473,Austin Reaves,107,27.244385,5.192888,5.937469,0.229487,0.128890,0.249237,25.085623,3.966913,4.147873
49,Anthony Davis,50,23.360000,10.740000,3.020000,0.207878,0.254299,0.113617,22.723537,7.826685,1.890849
0,LeBron James,90,14.424793,4.608175,6.948620,0.120850,0.115957,0.268296,13.210304,3.568873,4.465061
281,Deandre Ayton,27,13.776193,8.477551,0.813077,0.114788,0.211334,0.032330,12.547687,6.504323,0.538050
289,Rui Hachimura,99,12.645954,3.717454,0.600032,0.107942,0.093741,0.022677,11.799378,2.885122,0.377399
129,D'Angelo Russell,36,11.333333,2.527778,4.472222,0.103560,0.059579,0.172633,11.320344,1.833705,2.873010
555,Max Christie,53,8.075472,2.660377,1.320755,0.072572,0.062786,0.051116,7.932927,1.932400,0.850681
874,R.J. Davis,6,6.511482,1.200910,0.134488,0.068052,0.028401,0.005380,7.438880,0.874109,0.089528
614,Jake LaRavia,28,7.996439,3.504919,1.585511,0.067135,0.085651,0.066398,7.338672,2.636128,1.105008


In [10]:
# ==========================================
# CELL 9 – POISSON PROP SIMULATION HELPERS
# ==========================================

import numpy as np
import math
from typing import Optional, Dict, Any


def simulate_poisson_prop(
    lam: float,
    line: float,
    n_sims: int = 100_000,
    random_state: Optional[int] = None,
) -> Dict[str, Any]:
    """
    Simulate a single stat (points, rebounds, assists, etc.) as Poisson(lam)
    and estimate probabilities of going over/under a betting line.

    - lam:      expected value (your QEPC λ)
    - line:     betting line (e.g. 22.5, 8.5, 6.0, etc.)
    - n_sims:   number of simulated games
    """
    if lam <= 0 or not np.isfinite(lam):
        raise ValueError(f"Invalid lambda for Poisson: {lam}")

    rng = np.random.default_rng(random_state)
    samples = rng.poisson(lam, size=n_sims)

    if float(line).is_integer():
        line_int = int(line)
        over_mask = samples > line_int
        push_mask = samples == line_int
        under_mask = samples < line_int
        prob_push = push_mask.mean()
    else:
        # For half-lines (e.g. 22.5), push prob is zero
        over_mask = samples > line
        under_mask = samples < line
        prob_push = 0.0

    prob_over = over_mask.mean()
    prob_under = under_mask.mean()

    return {
        "lambda": float(lam),
        "line": float(line),
        "prob_over": float(prob_over),
        "prob_under": float(prob_under),
        "prob_push": float(prob_push),
        "mean_sim": float(samples.mean()),
        "std_sim": float(samples.std(ddof=0)),
        "n_sims": int(n_sims),
    }



In [11]:
# =========================================================
# CELL 10 – RUN PROPS FOR ONE PLAYER (PTS / REB / AST)
# =========================================================

# Choose which side to pull the player from:
#   - home_usage (home team)
#   - away_usage (away team)
target_df = home_usage  # or use away_usage

# Pick a player name from that team (adjust as needed)
# You can inspect:
#   target_df["player_name"].unique()
PLAYER_NAME = target_df["player_name"].iloc[0]  # change this to whoever you want

print("Using player:", PLAYER_NAME)

player_row = target_df[target_df["player_name"] == PLAYER_NAME]
if player_row.empty:
    raise ValueError(f"Player {PLAYER_NAME} not found in target_df.")

player_row = player_row.iloc[0]

# QEPC λs from your usage + team expectations
lam_pts = float(player_row["exp_points"])
lam_reb = float(player_row.get("exp_rebounds", np.nan))
lam_ast = float(player_row.get("exp_assists", np.nan))

print(f"\nQEPC λ for {PLAYER_NAME}:")
print(f"  Points λ = {lam_pts:.2f}")
print(f"  Rebounds λ = {lam_reb:.2f}")
print(f"  Assists λ = {lam_ast:.2f}")

# --- Set example betting lines (edit these to match a real book) ---

line_pts = round(lam_pts) + 0.5      # e.g. around his projection
line_reb = round(lam_reb) + 0.5 if np.isfinite(lam_reb) else None
line_ast = round(lam_ast) + 0.5 if np.isfinite(lam_ast) else None

print(f"\nExample lines (edit these):")
print(f"  Points line:   {line_pts}")
print(f"  Rebounds line: {line_reb}")
print(f"  Assists line:  {line_ast}")

# --- Simulate props using Poisson model ---

results = {}

results["points"] = simulate_poisson_prop(lam_pts, line_pts, n_sims=100_000)

if line_reb is not None and np.isfinite(lam_reb):
    results["rebounds"] = simulate_poisson_prop(lam_reb, line_reb, n_sims=100_000)

if line_ast is not None and np.isfinite(lam_ast):
    results["assists"] = simulate_poisson_prop(lam_ast, line_ast, n_sims=100_000)

print(f"\nSimulation results for {PLAYER_NAME}:")

for stat, res in results.items():
    print(f"\n[{stat.upper()}] line {res['line']} vs λ={res['lambda']:.2f}")
    print(f"  P(Over)  ≈ {res['prob_over']*100:5.2f}%")
    print(f"  P(Under) ≈ {res['prob_under']*100:5.2f}%")
    if res["prob_push"] > 0:
        print(f"  P(Push)  ≈ {res['prob_push']*100:5.2f}%")
    print(f"  Sim mean ≈ {res['mean_sim']:.2f} (simulated)")
    print(f"  Sim std  ≈ {res['std_sim']:.2f}")


Using player: Jaylen Brown

QEPC λ for Jaylen Brown:
  Points λ = 25.99
  Rebounds λ = 4.37
  Assists λ = 3.54

Example lines (edit these):
  Points line:   26.5
  Rebounds line: 4.5
  Assists line:  4.5

Simulation results for Jaylen Brown:

[POINTS] line 26.5 vs λ=25.99
  P(Over)  ≈ 44.69%
  P(Under) ≈ 55.31%
  Sim mean ≈ 25.99 (simulated)
  Sim std  ≈ 5.11

[REBOUNDS] line 4.5 vs λ=4.37
  P(Over)  ≈ 44.37%
  P(Under) ≈ 55.63%
  Sim mean ≈ 4.38 (simulated)
  Sim std  ≈ 2.09

[ASSISTS] line 4.5 vs λ=3.54
  P(Over)  ≈ 28.32%
  P(Under) ≈ 71.69%
  Sim mean ≈ 3.54 (simulated)
  Sim std  ≈ 1.88


In [12]:
# =========================================================
# CELL 11 – BATCH PROPS FOR MULTIPLE PLAYERS (POINTS ONLY)
# =========================================================

# You can choose home_usage or away_usage here
batch_df = home_usage  # or away_usage

# Define a dict of {player_name: points_line}
# (Replace with real players / lines as needed)
points_lines = {}

# Example: auto-generate 5 lines near their QEPC projections
for _, row in batch_df.head(5).iterrows():
    name = row["player_name"]
    lam_pts = float(row["exp_points"])
    # Example line = round λ + 0.5
    points_lines[name] = round(lam_pts) + 0.5

print("Batch points lines (edit this dict as needed):")
for name, line in points_lines.items():
    print(f"  {name}: {line}")

batch_results = []

for name, line in points_lines.items():
    row = batch_df[batch_df["player_name"] == name].iloc[0]
    lam_pts = float(row["exp_points"])

    res = simulate_poisson_prop(lam_pts, line, n_sims=100_000)
    batch_results.append(
        {
            "player_name": name,
            "line_pts": res["line"],
            "lambda_pts": res["lambda"],
            "prob_over": res["prob_over"],
            "prob_under": res["prob_under"],
            "prob_push": res["prob_push"],
            "mean_sim": res["mean_sim"],
            "std_sim": res["std_sim"],
        }
    )

batch_results_df = pd.DataFrame(batch_results)
batch_results_df = batch_results_df.sort_values("prob_over", ascending=False)

print("\nBatch results (sorted by P(Over)):")
display(batch_results_df)


Batch points lines (edit this dict as needed):
  Jaylen Brown: 26.5
  Jayson Tatum: 17.5
  Derrick White: 16.5
  Payton Pritchard: 16.5
  Kristaps Porzingis: 13.5

Batch results (sorted by P(Over)):


,player_name,line_pts,lambda_pts,prob_over,prob_under,prob_push,mean_sim,std_sim
2,Derrick White,16.5,16.284979,0.46161,0.53839,0.0,16.28542,4.018283
0,Jaylen Brown,26.5,25.989585,0.44723,0.55277,0.0,26.00315,5.106723
3,Payton Pritchard,16.5,15.638803,0.39936,0.60064,0.0,15.64071,3.967541
1,Jayson Tatum,17.5,16.541509,0.39251,0.60749,0.0,16.53899,4.066136
4,Kristaps Porzingis,13.5,12.545609,0.37720,0.62280,0.0,12.55169,3.544177


In [13]:
# =======================================================
# CELL 12 – RECENCY-WEIGHTED PLAYER POINTS (DECOHERENCE)
# =======================================================

from qepc.quantum.decoherence import recency_weighted_groupby_mean

# We assume you still have player_boxes_qepc loaded from earlier cells
# and that it has: player_id, team_name, game_id, game_date, points.

needed_cols = ["player_id", "team_name", "game_id", "game_date", "points"]
missing = [c for c in needed_cols if c not in player_boxes_qepc.columns]
if missing:
    raise ValueError(f"player_boxes_qepc is missing {missing}")

# Let's compute a *recency-weighted* average of points
# per (player_id, team_name), with a coherence time tau of 30 days.

tau_points_days = 30.0

rw_points = recency_weighted_groupby_mean(
    df=player_boxes_qepc,
    date_col="game_date",
    group_cols=["player_id", "team_name"],
    value_cols=["points"],
    tau_days=tau_points_days,
    ref_date=None,        # default: use max(game_date) as "today"
    clip_days=120.0,      # beyond 4 months, everything gets tiny weight
    weight_col_name="w_pts",
)

rw_points = rw_points.rename(columns={"points": "rw_avg_points"})

print("Recency-weighted points per player/team (sample):")
display(rw_points.head(20))


Recency-weighted points per player/team (sample):


,player_id,team_name,total_weight,rw_avg_points
0,2,Grizzlies,1.465251,10.237500
1,2,Lakers,18.315639,15.003000
2,2,Pacers,3.296815,9.588889
3,3,Celtics,0.989044,1.370370
4,3,Grizzlies,3.608181,5.157360
5,3,Hawks,4.267544,11.802575
6,3,Heat,8.773191,11.605428
7,3,Pistons,2.326086,3.874016
8,7,Bucks,4.047756,6.828054
9,7,Heat,0.622732,3.176471


In [14]:
# ==========================================================
# CELL 13 – ENTROPY OF A PLAYER'S POINTS DISTRIBUTION
# ==========================================================

import numpy as np
from qepc.quantum.entropy import sample_entropy

# We'll use the same λ you used in Cell 10 for PLAYER_NAME
lam_for_entropy = lam_pts  # from Cell 10

n_sims_entropy = 100_000
rng = np.random.default_rng(12345)

samples_pts = rng.poisson(lam_for_entropy, size=n_sims_entropy)

H_bits, pmf = sample_entropy(samples_pts, base=2.0, return_pmf=True)

print(f"Entropy for {PLAYER_NAME}'s points distribution:")
print(f"  H ≈ {H_bits:.3f} bits (0 = deterministic, higher = more spread)")

# Optional: show a quick table of outcomes near the peak
import pandas as pd

values = np.arange(len(pmf))
mask = pmf > 0
df_pmf = pd.DataFrame({"points": values[mask], "prob": pmf[mask]})
df_pmf = df_pmf[
    (df_pmf["points"] >= lam_for_entropy - 10)
    & (df_pmf["points"] <= lam_for_entropy + 10)
]

print("\nLocal view of the simulated PMF around λ:")
display(df_pmf.head(25))


Entropy for Jaylen Brown's points distribution:
  H ≈ 3.864 bits (0 = deterministic, higher = more spread)

Local view of the simulated PMF around λ:


,points,prob
2,3,0.00126
3,4,0.00372
4,5,0.00933
5,6,0.01955
6,7,0.03454
7,8,0.05487
8,9,0.07833
9,10,0.09335
10,11,0.10735
11,12,0.11177


In [15]:
# ==========================================================
# CELL 13 – BATCH PROPS WITH EDGE + ENTROPY (POINTS)
# ==========================================================

import numpy as np
import pandas as pd
from qepc.quantum.entropy import sample_entropy

# Choose which team to analyze
batch_df = home_usage  # or away_usage

# Define lines: either auto-near-λ or hand-edit later
points_lines = {
    "Jayson Tatum": 28.5,
    "Jaylen Brown": 23.5,
    "Derrick White": 15.5,
}


for _, row in batch_df.head(8).iterrows():  # take first 8 players for example
    name = row["player_name"]
    lam_pts_i = float(row["exp_points"])
    # Example: line = round λ + 0.5 (just a placeholder)
    points_lines[name] = round(lam_pts_i) + 0.5

print("Points lines used in this batch (edit these as you like):")
for name, line in points_lines.items():
    print(f"  {name}: {line}")

rng = np.random.default_rng(777)
n_sims = 50_000  # fewer is fine for batch

rows = []

for name, line in points_lines.items():
    row = batch_df[batch_df["player_name"] == name].iloc[0]
    lam = float(row["exp_points"])

    # 1) Poisson prop simulation result (P over/under)
    sim_result = simulate_poisson_prop(lam, line, n_sims=n_sims, random_state=42)

    # 2) Raw samples for entropy
    samples = rng.poisson(lam, size=n_sims)
    H_bits, _ = sample_entropy(samples, base=2.0, return_pmf=False)

    rows.append(
        {
            "player_name": name,
            "lambda_pts": lam,
            "line_pts": sim_result["line"],
            "prob_over": sim_result["prob_over"],
            "prob_under": sim_result["prob_under"],
            "prob_push": sim_result["prob_push"],
            "mean_sim": sim_result["mean_sim"],
            "std_sim": sim_result["std_sim"],
            "entropy_bits": H_bits,
        }
    )

batch_results_df = pd.DataFrame(rows)

# Edge = model's over prob minus a neutral 50% (for half lines)
batch_results_df["edge_over_pct"] = (batch_results_df["prob_over"] - 0.5) * 100.0

# Sort by some combination: big edge, lower entropy first
batch_results_df = batch_results_df.sort_values(
    ["edge_over_pct", "entropy_bits"], ascending=[False, True]
)

print("\nBatch props with edge + entropy (points):")
display(
    batch_results_df[
        [
            "player_name",
            "lambda_pts",
            "line_pts",
            "prob_over",
            "prob_under",
            "edge_over_pct",
            "entropy_bits",
            "mean_sim",
            "std_sim",
        ]
    ]
)


Points lines used in this batch (edit these as you like):
  Jayson Tatum: 17.5
  Jaylen Brown: 26.5
  Derrick White: 16.5
  Payton Pritchard: 16.5
  Kristaps Porzingis: 13.5
  Anfernee Simons: 12.5
  Neemias Queta: 9.5
  Jrue Holiday: 9.5

Batch props with edge + entropy (points):


,player_name,lambda_pts,line_pts,prob_over,prob_under,edge_over_pct,entropy_bits,mean_sim,std_sim
2,Derrick White,16.284979,16.5,0.46228,0.53772,-3.772,4.044237,16.28198,4.012798
1,Jaylen Brown,25.989585,26.5,0.44722,0.55278,-5.278,4.391547,25.98194,5.072811
5,Anfernee Simons,12.046095,12.5,0.42962,0.57038,-7.038,3.831166,12.04680,3.453689
3,Payton Pritchard,15.638803,16.5,0.39882,0.60118,-10.118,4.023763,15.63540,3.937698
0,Jayson Tatum,16.541509,17.5,0.39256,0.60744,-10.744,4.056759,16.53536,4.044605
4,Kristaps Porzingis,12.545609,13.5,0.37622,0.62378,-12.378,3.864132,12.53760,3.532702
6,Neemias Queta,8.679288,9.5,0.36842,0.63158,-13.158,3.588325,8.66834,2.949275
7,Jrue Holiday,8.605042,9.5,0.35924,0.64076,-14.076,3.582307,8.60090,2.938663


In [16]:
# ==========================================================
# CELL 14 – TEAM ENTANGLEMENT (POINTS) FOR HOME TEAM
# ==========================================================

from qepc.quantum.entanglement import build_team_entanglement

print("Home team for this game:", home_team)

ent_points_home = build_team_entanglement(
    player_boxes=player_boxes_qepc,
    team_name=home_team,
    stat_col="points",
    min_shared_games=10,        # can adjust
    date_col="game_date",
    cutoff_date="2024-10-01",   # focus on recent-ish history
)

print(f"Entanglement table for {home_team} (points) – sample:")
display(ent_points_home.head(20))


Home team for this game: Celtics


C:\Users\wdors\qepc_project\qepc\quantum\entanglement.py:155: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  corr_long = corr_mat.stack(dropna=False).reset_index(name="corr")


ValueError: cannot insert player_id_b, already exists

In [ ]:
# =================================================================
# CELL 15 – STAR-FOCUSED ENTANGLEMENT VIEW (POINTS, HOME TEAM)
# =================================================================

from qepc.quantum.entanglement import get_player_entanglement_view

# Make sure we're looking at the same team's usage as ent_points_home
# (i.e., target_df in Cell 10 should be home_usage for this to match)
star_name = PLAYER_NAME
print("Current star (from props cell):", star_name)

# Get star's player_id from home_usage
star_row = home_usage[home_usage["player_name"] == star_name]
if star_row.empty:
    raise ValueError(f"PLAYER_NAME={star_name!r} not found in home_usage. "
                     "Make sure target_df in Cell 10 is home_usage.")

star_id = int(star_row["player_id"].iloc[0])

print(f"Star player_id: {star_id}")

star_ent = get_player_entanglement_view(
    ent_df=ent_points_home,
    player_id=star_id,
    min_shared_games=10,
)

if star_ent.empty:
    print("No entanglement pairs found for this player with sufficient shared games.")
else:
    print(f"\nTop positively entangled teammates with {star_name} (points):")
    display(star_ent[star_ent["corr"] > 0].head(10))

    print(f"\nTop negatively entangled teammates with {star_name} (points):")
    display(star_ent[star_ent["corr"] < 0].head(10))


In [ ]:
# ==========================================================
# CELL 16 – DEFINE SGP PICKS (POINTS, SAME GAME)
# ==========================================================

from qepc.quantum.correlated_sim import PlayerPropConfig

# Choose which side to work with
sgp_home = True   # set False if you want away team instead

if sgp_home:
    sgp_df = home_usage
    team_label = home_team
else:
    sgp_df = away_usage
    team_label = away_team

print(f"Building SGP for team: {team_label}")

# See available names quickly (optional)
print("\nSample of available players on this team:")
display(sgp_df[["player_id", "player_name", "avg_points", "exp_points"]].head(15))

# ---- DEFINE YOUR SGP PLAYERS + LINES HERE ----
# You can either:
#   1) Hardcode a dict of {player_name: line_pts}, or
#   2) Start from exp_points and tweak.

# Example: auto-pick top 3 scorers by exp_points and set line near λ
top_players = sgp_df.sort_values("exp_points", ascending=False).head(3)

sgp_lines = {}
for _, row in top_players.iterrows():
    name = row["player_name"]
    lam_pts = float(row["exp_points"])
    # Example line = round λ + 0.5
    sgp_lines[name] = round(lam_pts) + 0.5

print("\nInitial SGP lines (edit this dict if you want specific lines):")
for name, line in sgp_lines.items():
    print(f"  {name}: {line}")

# If you want to override with real lines, you can do something like:
# sgp_lines = {
#     "Jayson Tatum": 28.5,
#     "Jaylen Brown": 22.5,
#     "Derrick White": 14.5,
# }

# Build PlayerPropConfig list
sgp_players: list[PlayerPropConfig] = []

for name, line in sgp_lines.items():
    row = sgp_df[sgp_df["player_name"] == name]
    if row.empty:
        raise ValueError(f"SGP player {name!r} not found in usage table for {team_label}.")
    row = row.iloc[0]
    lam_pts = float(row["exp_points"])
    sgp_players.append(
        PlayerPropConfig(
            player_id=int(row["player_id"]),
            player_name=name,
            lambda_pts=lam_pts,
            line_pts=float(line),
        )
    )

print(f"\nConfigured {len(sgp_players)} players for entangled SGP sim.")
for p in sgp_players:
    print(f"  - {p.player_name}: λ={p.lambda_pts:.2f}, line={p.line_pts}")


In [ ]:
# ==========================================================
# CELL 17 – RUN ENTANGLED SGP SIM (POINTS, SAME TEAM)
# ==========================================================

from qepc.quantum.correlated_sim import simulate_entangled_points

# Hyperparameters for volatility (tweak if you want)
N_SIMS = 100_000
TEAM_SIGMA = 0.25   # team-level volatility (all players share)
PLAYER_SIGMA = 0.15 # individual volatility

sgp_result = simulate_entangled_points(
    players=sgp_players,
    n_sims=N_SIMS,
    random_state=2025,
    team_sigma=TEAM_SIGMA,
    player_sigma=PLAYER_SIGMA,
)

marginals_df = sgp_result["marginals"]
joint_info = sgp_result["joint"]
pairwise_corr = sgp_result["pairwise_corr"]

print(f"Entangled SGP sim with {joint_info['n_sims']} universes for team {team_label}.\n")

print("Per-player marginal props (from correlated sim):")
display(
    marginals_df[
        [
            "player_name",
            "lambda_pts",
            "line_pts",
            "prob_over",
            "prob_under",
            "mean_sim",
            "std_sim",
        ]
    ]
)

print("\nJoint SGP probabilities (all players' points props):")
print(f"  Naive product P(all overs)   ≈ {joint_info['naive_product_all_over']*100:5.2f}%")
print(f"  Entangled P(all overs)       ≈ {joint_info['prob_all_over']*100:5.2f}%")
print(f"  P(at least one over)         ≈ {joint_info['prob_any_over']*100:5.2f}%")
print(f"  P(none go over)              ≈ {joint_info['prob_none_over']*100:5.2f}%")


In [ ]:
# ==========================================================
# CELL 18 – EMPIRICAL CORRELATIONS BETWEEN SGP PLAYERS
# ==========================================================

print("Empirical correlations between simulated points for SGP players:")
display(pairwise_corr)
